In [1]:
import numpy as np
import keras
from keras.models import Model
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import pandas as pd

Using TensorFlow backend.
/home/spynal/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
IMG_SIZE = (299, 299)  # размер входного изображения сети

In [3]:
transformation_ratio = .05

In [4]:
train_dir = '/home/spynal/Netology/Data/CvD/train/'
val_dir = '/home/spynal/Netology/Data/CvD/val/'
test_dir = '/home/spynal/Netology/Data/CvD/test/'

In [6]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size=(299, 299),
                                                        batch_size=32,
                                                        class_mode='binary')


validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                                  target_size=(299, 299),
                                                                  batch_size=32,
                                                                  class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                                  target_size=(299, 299),
                                                                  batch_size=1,
                                                                  shuffle = False,
                                                                  class_mode='binary')


Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [12]:
base_model = Xception(include_top = False,
                   weights = 'imagenet',
                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3))

In [4]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [14]:
for layer in base_model.layers:
    layer.trainable = False

In [15]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',  # функция потерь binary_crossentropy (log loss
              metrics=['accuracy'])

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [21]:
# запускаем процесс обучения
model.fit_generator(train_generator,  # данные читаем функцией-генератором
        steps_per_epoch=10,  # число вызовов генератора за эпоху
        epochs=100,  # число эпох обучения
        validation_data=validation_generator)

Epoch 1/100
10/10 [==============================] - 35s 3s/step - loss: 0.5128 - acc: 0.7562 - val_loss: 0.1530 - val_acc: 0.9560
Epoch 2/100
10/10 [==============================] - 29s 3s/step - loss: 0.3833 - acc: 0.8469 - val_loss: 0.1912 - val_acc: 0.9110
Epoch 3/100
10/10 [==============================] - 29s 3s/step - loss: 0.2106 - acc: 0.8844 - val_loss: 0.0674 - val_acc: 0.9810
Epoch 4/100
10/10 [==============================] - 29s 3s/step - loss: 0.2103 - acc: 0.9000 - val_loss: 0.0755 - val_acc: 0.9790
Epoch 5/100
10/10 [==============================] - 29s 3s/step - loss: 0.1896 - acc: 0.9375 - val_loss: 0.1470 - val_acc: 0.9480
Epoch 6/100
10/10 [==============================] - 29s 3s/step - loss: 0.1567 - acc: 0.9375 - val_loss: 0.1100 - val_acc: 0.9660
Epoch 7/100
10/10 [==============================] - 29s 3s/step - loss: 0.1236 - acc: 0.9313 - val_loss: 0.1588 - val_acc: 0.9350
Epoch 8/100
10/10 [==============================] - 29s 3s/step - loss: 0.1532 - a

In [24]:
pred = model.predict_generator(test_generator, 12500, max_queue_size=500, verbose = 1)

12500/12500 [==============================] - 490s 39ms/step


In [25]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(20, 20))
for i, (path, score) in enumerate(zip(test_files[:4], pred[:4]), 1):
    subplot = fig.add_subplot(i // 5 + 1, 5, i)
    plt.imshow(plt.imread(path));
    subplot.set_title('%.3f' % score);

NameError: name 'test_files' is not defined

In [11]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [12]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True



In [13]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

In [14]:
model.fit_generator(fit_generator(train_files[train_val_split:]),  # данные читаем функцией-генератором
        steps_per_epoch=10,  # число вызовов генератора за эпоху
        epochs=100,  # число эпох обучения
        validation_data=validation_data)

Epoch 1/100
10/10 [==============================] - 9s 911ms/step - loss: 0.0975 - val_loss: 0.0087
Epoch 2/100
10/10 [==============================] - 7s 678ms/step - loss: 0.0597 - val_loss: 0.0091
Epoch 3/100
10/10 [==============================] - 7s 679ms/step - loss: 0.0790 - val_loss: 0.0088
Epoch 4/100
10/10 [==============================] - 7s 678ms/step - loss: 0.0592 - val_loss: 0.0084
Epoch 5/100
10/10 [==============================] - 7s 676ms/step - loss: 0.0568 - val_loss: 0.0093
Epoch 6/100
10/10 [==============================] - 7s 675ms/step - loss: 0.0380 - val_loss: 0.0095
Epoch 7/100
10/10 [==============================] - 7s 676ms/step - loss: 0.0631 - val_loss: 0.0092
Epoch 8/100
10/10 [==============================] - 7s 674ms/step - loss: 0.1080 - val_loss: 0.0099
Epoch 9/100
10/10 [==============================] - 7s 674ms/step - loss: 0.0467 - val_loss: 0.0095
Epoch 10/100
10/10 [==============================] - 7s 675ms/step - loss: 0.0700 - val_lo

In [26]:
with open('submit.txt', 'w') as dst:
    dst.write('id,label\n')
    for path, score in zip(test_files, pred):
        dst.write('%s,%f\n' % (re.search('(\d+)', path).group(0), score))